# Google Civic API
https://developers.google.com/civic-information/docs/using_api

In [2]:
import json
import os
import requests

In [3]:
# get a key at https://console.developers.google.com/apis/credentials
google_api_key = json.load(open("../private/google-civic-api.json"))["api_key"]

In [4]:
google_session = requests.Session()
google_session.params.update({"key": google_api_key})

## Elections

In [5]:
elections_resp = google_session.get("https://www.googleapis.com/civicinfo/v2/elections")
elections_resp.raise_for_status()
elections = elections_resp.json()

In [6]:
elections.keys()

dict_keys(['elections', 'kind'])

In [7]:
len(elections["elections"])

10

In [8]:
elections["elections"]

[{'id': '2000',
  'name': 'VIP Test Election',
  'electionDay': '2021-06-06',
  'ocdDivisionId': 'ocd-division/country:us'},
 {'id': '4953',
  'name': 'Louisiana Municipal General Election',
  'electionDay': '2020-08-15',
  'ocdDivisionId': 'ocd-division/country:us/state:la'},
 {'id': '4979',
  'name': 'Georgia General Primary Runoff Election',
  'electionDay': '2020-08-11',
  'ocdDivisionId': 'ocd-division/country:us/state:ga'},
 {'id': '5003',
  'name': 'Michigan State Primary Election',
  'electionDay': '2020-08-04',
  'ocdDivisionId': 'ocd-division/country:us/state:mi'},
 {'id': '5005',
  'name': 'Washington State Primary Election',
  'electionDay': '2020-08-04',
  'ocdDivisionId': 'ocd-division/country:us/state:wa'},
 {'id': '5007',
  'name': 'Minnesota State Primary Election',
  'electionDay': '2020-08-11',
  'ocdDivisionId': 'ocd-division/country:us/state:mn'},
 {'id': '5008',
  'name': 'Vermont State Primary Election',
  'electionDay': '2020-08-11',
  'ocdDivisionId': 'ocd-divi

## VoterInfo

In [33]:
# N.B. I'm using 2000 from the demo
# The docs imply that electionId should be determined by a prior query to the elections
# endpoint. 
election_id = 2000

voterinfo_resp = google_session.get("https://www.googleapis.com/civicinfo/v2/voterinfo",
                                    params={"address":"1 Sussex St., 94131",
                                            "electionId": election_id
                                           })
voterinfo_resp.raise_for_status()
voterinfo = voterinfo_resp.json()

In [18]:
sorted(voterinfo.keys())

['contests',
 'election',
 'kind',
 'normalizedInput',
 'pollingLocations',
 'state']

### Transparent responses: election, kind, normalizedInput, pollingLocations, state

In [14]:
voterinfo["election"]

{'id': '2000',
 'name': 'VIP Test Election',
 'electionDay': '2021-06-06',
 'ocdDivisionId': 'ocd-division/country:us'}

In [19]:
voterinfo["kind"]

'civicinfo#voterInfoResponse'

In [11]:
voterinfo["normalizedInput"]

{'line1': '1 Sussex Street',
 'city': 'San Francisco',
 'state': 'CA',
 'zip': '94131'}

In [12]:
voterinfo["pollingLocations"]

[{'address': {'locationName': 'Garage',
   'line1': '742 Chenery St',
   'city': 'San Francisco',
   'state': 'CA',
   'zip': '94131-2907'},
  'notes': 'Btw Lippard & Brompton Ave',
  'pollingHours': '07:00-20:00',
  'sources': [{'name': 'Voting Information Project', 'official': True}]}]

In [13]:
voterinfo["state"]

[{'name': 'California',
  'electionAdministrationBody': {'name': 'California Secretary of State',
   'electionInfoUrl': 'http://www.sos.ca.gov/elections',
   'votingLocationFinderUrl': 'https://www.sos.ca.gov/elections/polling-place/',
   'ballotInfoUrl': 'https://www.sos.ca.gov/elections/polling-place/',
   'correspondenceAddress': {'line1': '1500 11th Street',
    'city': 'Sacramento',
    'state': 'California',
    'zip': '95814-5701'}},
  'local_jurisdiction': {'name': 'San Francisco County',
   'electionAdministrationBody': {'electionInfoUrl': 'http://sfelections.org',
    'physicalAddress': {'line1': 'CITY HALL - 1 DR. CARLTON B GOODLETT PLACE, ROOM 48',
     'city': 'SAN FRANCISCO',
     'state': 'CA',
     'zip': '94102-4635'},
    'electionOfficials': [{'officePhoneNumber': '(415) 554-4375',
      'emailAddress': 'sfvote@sfgov.org'}]},
   'sources': [{'name': 'DemocracyWorks', 'official': False}]},
  'sources': [{'name': '', 'official': False}]}]

### contests

In [20]:
voterinfo["contests"]

[{'type': 'General',
  'office': 'US House - District 12',
  'level': ['country'],
  'roles': ['legislatorLowerBody'],
  'district': {'name': "California's 12th congressional district",
   'scope': 'congressional',
   'id': 'ocd-division/country:us/state:ca/cd:12'},
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'candidates': [{'name': 'Nancy Pelosi', 'party': 'Democratic'},
   {'name': 'John Dennis',
    'party': 'Republican',
    'candidateUrl': 'http://www.johndennisforcongress.com/',
    'channels': [{'type': 'Facebook',
      'id': 'https://www.facebook.com/johndennis2010'},
     {'type': 'Twitter', 'id': 'https://twitter.com/johndennis2012'}]}]},
 {'type': 'General',
  'office': 'Governor',
  'level': ['administrativeArea1'],
  'roles': ['headOfGovernment'],
  'district': {'name': 'California',
   'scope': 'statewide',
   'id': 'ocd-division/country:us/state:ca'},
  'sources': [{'name': 'Ballot Information Project', 'official': False}],
  'candidates'

In [21]:
len(voterinfo["contests"])

20

In [23]:
from IPython.display import HTML, display
import tabulate
def display_table(data):
    display(HTML(tabulate.tabulate(data, tablefmt='html')))

In [28]:
display_table([(c["type"], c.get("office"), c["district"]["id"]) for c in voterinfo["contests"]])

General,US House - District 12,ocd-division/country:us/state:ca/cd:12
General,Governor,ocd-division/country:us/state:ca
General,Lieutenant Governor,ocd-division/country:us/state:ca
General,Controller,ocd-division/country:us/state:ca
General,Secretary of State,ocd-division/country:us/state:ca
General,Attorney General,ocd-division/country:us/state:ca
General,Treasurer,ocd-division/country:us/state:ca
General,Superintendent of Public Instruction,ocd-division/country:us/state:ca
General,Insurance Commissioner,ocd-division/country:us/state:ca
General,Supreme Court Associate Justice (Retention),ocd-division/country:us/state:ca
General,State Assembly - District 17,ocd-division/country:us/state:ca/sldl:17
